In [46]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import os

In [53]:
def create_model():
    model = Sequential([
        Conv2D(128, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(32, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation='relu'),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [51]:
# Load dataset
df = pd.read_csv('/kaggle/input/cat-dog-images-for-classification/cat_dog.csv')
df['labels'] = df['labels'].apply(lambda x: 'cat' if x == 0 else 'dog')

# Split the dataset
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# Image Data Generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    directory='/kaggle/input/cat-dog-images-for-classification/cat_dog', 
    x_col='image', 
    y_col='labels', 
    target_size=(128, 128), 
    class_mode='binary', 
    batch_size=32
)

val_generator = val_datagen.flow_from_dataframe(
    val_df, 
    directory='/kaggle/input/cat-dog-images-for-classification/cat_dog', 
    x_col='image', 
    y_col='labels', 
    target_size=(128, 128), 
    class_mode='binary', 
    batch_size=32
)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.


In [54]:
model = create_model()
history = model.fit(
    train_generator, 
    epochs=10, 
    validation_data=val_generator
)

Epoch 1/10
  4/625 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.5215 - loss: 0.7840

W0000 00:00:1721862212.941243     132 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.5469 - loss: 0.6870

W0000 00:00:1721862252.784387     135 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


625/625 ━━━━━━━━━━━━━━━━━━━━ 54s 79ms/step - accuracy: 0.5470 - loss: 0.6870 - val_accuracy: 0.6344 - val_loss: 0.6447
Epoch 2/10


W0000 00:00:1721862262.201905     132 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


625/625 ━━━━━━━━━━━━━━━━━━━━ 50s 79ms/step - accuracy: 0.6636 - loss: 0.6147 - val_accuracy: 0.6758 - val_loss: 0.5934
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 49s 78ms/step - accuracy: 0.7394 - loss: 0.5249 - val_accuracy: 0.7526 - val_loss: 0.5105
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 50s 79ms/step - accuracy: 0.8081 - loss: 0.4202 - val_accuracy: 0.7428 - val_loss: 0.5371
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 50s 80ms/step - accuracy: 0.8796 - loss: 0.2861 - val_accuracy: 0.7646 - val_loss: 0.5979
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 49s 78ms/step - accuracy: 0.9446 - loss: 0.1431 - val_accuracy: 0.7504 - val_loss: 0.9395
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 49s 77ms/step - accuracy: 0.9701 - loss: 0.0853 - val_accuracy: 0.7482 - val_loss: 1.0050
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 49s 77ms/step - accuracy: 0.9875 - loss: 0.0387 - val_accuracy: 0.7422 - val_loss: 1.4716
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 49s 77ms/step - accuracy: 0.9927 - loss: 0.0277 - val_accurac

In [55]:
# Evaluate the model
loss, accuracy = model.evaluate(val_generator)
print(f'Accuracy on raw images: {accuracy}')

157/157 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - accuracy: 0.7468 - loss: 1.3687
Accuracy on raw images: 0.7414000034332275


# ----------------------- After Prerocessing ---------------------

In [56]:
def apply_filters(image):
    # Convert to array
    image = np.array(image)
    
    # Apply a sharpening filter
    kernel = np.array([[0, -1, 0], 
                       [-1, 5,-1],
                       [0, -1, 0]])
    image = cv2.filter2D(src=image, ddepth=-1, kernel=kernel)
    
    return image

# Image Data Generators with augmentation and custom filter
train_datagen_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=apply_filters
)

train_generator_aug = train_datagen_aug.flow_from_dataframe(
    train_df, 
    directory='/kaggle/input/cat-dog-images-for-classification/cat_dog', 
    x_col='image', 
    y_col='labels', 
    target_size=(128, 128), 
    class_mode='binary', 
    batch_size=32
)

Found 20000 validated image filenames belonging to 2 classes.


In [57]:
# Create and train the model on augmented data
model_aug = create_model()
history_aug = model_aug.fit(
    train_generator_aug, 
    epochs=10, 
    validation_data=val_generator
)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


  1/625 ━━━━━━━━━━━━━━━━━━━━ 54:47 5s/step - accuracy: 0.5938 - loss: 0.7052

W0000 00:00:1721862783.057637     132 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 194ms/step - accuracy: 0.5433 - loss: 0.6862

W0000 00:00:1721862904.614511     135 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


625/625 ━━━━━━━━━━━━━━━━━━━━ 136s 210ms/step - accuracy: 0.5434 - loss: 0.6861 - val_accuracy: 0.6232 - val_loss: 0.6531
Epoch 2/10


W0000 00:00:1721862914.111829     133 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


625/625 ━━━━━━━━━━━━━━━━━━━━ 132s 209ms/step - accuracy: 0.6847 - loss: 0.5932 - val_accuracy: 0.5638 - val_loss: 0.7316
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 132s 209ms/step - accuracy: 0.7674 - loss: 0.4847 - val_accuracy: 0.6658 - val_loss: 0.5929
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 134s 213ms/step - accuracy: 0.8021 - loss: 0.4381 - val_accuracy: 0.6790 - val_loss: 0.5865
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 134s 213ms/step - accuracy: 0.8060 - loss: 0.4225 - val_accuracy: 0.6332 - val_loss: 0.6561
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 133s 211ms/step - accuracy: 0.8271 - loss: 0.3896 - val_accuracy: 0.6752 - val_loss: 0.5973
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 133s 210ms/step - accuracy: 0.8415 - loss: 0.3614 - val_accuracy: 0.7294 - val_loss: 0.5390
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 134s 212ms/step - accuracy: 0.8466 - loss: 0.3543 - val_accuracy: 0.6502 - val_loss: 0.6723
Epoch 9/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 133s 210ms/step - accuracy: 0.8548 - loss: 0.33

In [58]:
loss_aug, accuracy_aug = model_aug.evaluate(val_generator)
print(f'Accuracy on processed images: {accuracy_aug}')

157/157 ━━━━━━━━━━━━━━━━━━━━ 9s 60ms/step - accuracy: 0.7401 - loss: 0.5038
Accuracy on processed images: 0.7396000027656555


In [59]:
print(f'Accuracy on raw images: {accuracy}')
print(f'Accuracy on processed images: {accuracy_aug}')

Accuracy on raw images: 0.7414000034332275
Accuracy on processed images: 0.7396000027656555


From output 
- actual image without any preprocessing is high accuracy than processed image.
because actual image high quailty and don't need to processed.